In [1]:
import numpy as np
import pandas as pd

from ripser import ripser, Rips
from persim import plot_diagrams, persistent_entropy, sliced_wasserstein, bottleneck, bottleneck_matching
from gtda.time_series import SingleTakensEmbedding
from gtda.plotting import plot_point_cloud
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy, BettiCurve, PersistenceImage
from gtda.diagrams import Amplitude, NumberOfPoints

from sklearn.pipeline import make_union
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

from os.path import join
import time

import matplotlib.pyplot as plt

In [89]:
# Feature analysis for homological dimension 1
hdim = 1

In [2]:
# Load data
data = pd.read_csv('preproc_data_/preprocessed.csv')

### Inbalance 

In [3]:
healthy = data[data['target']=='healthy'].sample(n=76, random_state=1)
trauma = data[data['target']=='trauma']

In [4]:
data = healthy.append(trauma)

### Embedding

In [5]:
max_embedding_dimension = 10
max_time_delay = 10
stride = 3

embedder = SingleTakensEmbedding(
    parameters_type="search", time_delay=max_time_delay, dimension=max_embedding_dimension,
)

In [6]:
def fit_embedder(embedder: SingleTakensEmbedding, y: np.ndarray, verbose: bool=True) -> np.ndarray:
    """Fits a Takens embedder and displays optimal search parameters."""
    y_embedded = embedder.fit_transform(y)

    if verbose:
        print(f"Shape of embedded time series: {y_embedded.shape}")
        print(
            f"Optimal embedding dimension is {embedder.dimension_} and time delay is {embedder.time_delay_}"
        )

    return y_embedded

### Point Cloud

In [7]:
file = data['fn']

point_cloud = []

tic = time.time()
for i, f in enumerate(file):
    file_path = join('preproc_data_', f)
    signal = pd.read_csv(file_path)
    
    embedding = []
    
    for j in range(1, signal.shape[1]):
        feati_embedded = fit_embedder(embedder, signal.iloc[:,j], False)
        feati_embedded = feati_embedded[None, :, :]
        embedding.append(feati_embedded)
    
    point_cloud.append(embedding)

toc = time.time()

timelapse = toc - tic
print("Elapsed Time: %.3g seconds" % (timelapse))

Elapsed Time: 104 seconds


### Persistence Diagram

In [8]:
def persistent_homology(fembedding, hdim):
    homology_dimensions = [hdim]
    
    persistence = VietorisRipsPersistence(homology_dimensions=homology_dimensions, n_jobs=-1)
    diagram = persistence.fit_transform(fembedding)
    
    return diagram

In [10]:
n = len(point_cloud) # number of subjects
m = len(point_cloud[0]) # number of channels (original features)

hdimension = hdim
diagrams = []

tic = time.time()
for i in range(n):
    
    print("Processing subject: ", i)
    diagram_ = []
    
    for j in range(m):
        diagram_.append(persistent_homology(point_cloud[i][j], hdimension))
    
    diagrams.append(diagram_)

toc = time.time()

timelapse = toc - tic
print("Elapsed Time: %.3g seconds" % (timelapse))

Processing subject:  0
Processing subject:  1
Processing subject:  2
Processing subject:  3
Processing subject:  4
Processing subject:  5
Processing subject:  6
Processing subject:  7
Processing subject:  8
Processing subject:  9
Processing subject:  10
Processing subject:  11
Processing subject:  12
Processing subject:  13
Processing subject:  14
Processing subject:  15
Processing subject:  16
Processing subject:  17
Processing subject:  18
Processing subject:  19
Processing subject:  20
Processing subject:  21
Processing subject:  22
Processing subject:  23
Processing subject:  24
Processing subject:  25
Processing subject:  26
Processing subject:  27
Processing subject:  28
Processing subject:  29
Processing subject:  30
Processing subject:  31
Processing subject:  32
Processing subject:  33
Processing subject:  34
Processing subject:  35
Processing subject:  36
Processing subject:  37
Processing subject:  38
Processing subject:  39
Processing subject:  40
Processing subject:  41
Pr

### Feature Extraction

In [23]:
def extract_features(diagram):
    
    # Metrics to calculate amplitudes
    metrics = [
        {"metric": metric}
        for metric in ["bottleneck", "wasserstein", "landscape", "persistence_image"]
    ]

    # Generation of topological features
    topological_feature = make_union(
        PersistenceEntropy(normalize=True),
        NumberOfPoints(n_jobs=-1),
        *[Amplitude(**metric, n_jobs=-1) for metric in metrics]
    )

    feature_i = topological_feature.fit_transform(diagram)
    
    return feature_i

* Feature extraction from 1 Homological dimension

In [32]:
features = []

tic = time.time()
for i in range(n):
    
    print("Extracting features for subject: ", i)
    feature_ = extract_features(diagrams[i][0])
    
    for j in range(1, m):
        feature_ = np.append(feature_, extract_features(diagrams[i][j]))
    
    features.append(feature_)

toc = time.time()

timelapse = toc - tic
print("Elapsed Time: %.3g seconds" % (timelapse))

Extracting features for subject:  0
Extracting features for subject:  1
Extracting features for subject:  2
Extracting features for subject:  3
Extracting features for subject:  4
Extracting features for subject:  5
Extracting features for subject:  6
Extracting features for subject:  7
Extracting features for subject:  8
Extracting features for subject:  9
Extracting features for subject:  10
Extracting features for subject:  11
Extracting features for subject:  12
Extracting features for subject:  13
Extracting features for subject:  14
Extracting features for subject:  15
Extracting features for subject:  16
Extracting features for subject:  17
Extracting features for subject:  18
Extracting features for subject:  19
Extracting features for subject:  20
Extracting features for subject:  21
Extracting features for subject:  22
Extracting features for subject:  23
Extracting features for subject:  24
Extracting features for subject:  25
Extracting features for subject:  26
Extracting 

In [80]:
df = pd.DataFrame(features)

In [81]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,92,93,94,95,96,97,98,99,100,101
0,1.057422,516.0,1.078493,7.609457,1.625817,240.086791,1.040205,510.0,1.458287,8.327879,...,1.759852,9.577967,3.196466,210.555888,0.991431,492.0,1.875970,10.882537,3.736711,182.475285
1,0.944617,503.0,1.944156,14.714389,4.243747,170.602065,1.018074,496.0,1.396510,9.424052,...,0.957242,7.267961,1.381102,257.864981,1.066423,463.0,1.016243,7.024251,1.655713,235.886002
2,1.001907,400.0,2.265818,9.232568,4.269111,150.670842,1.036141,401.0,1.721094,7.744508,...,1.241644,4.538302,1.427395,358.169170,1.117105,472.0,1.178067,5.557385,1.762974,264.469598
3,0.892047,442.0,3.056262,19.356665,7.804968,98.733253,0.979256,418.0,1.833882,10.771632,...,6.829617,36.005833,20.342606,68.578706,0.831882,431.0,7.570616,30.930352,24.124375,69.595830
4,1.005080,538.0,1.528481,10.438202,3.505291,194.476743,1.017692,532.0,1.621855,9.688185,...,2.766696,14.993556,6.180069,101.878752,0.939892,486.0,1.932256,14.908890,4.437211,140.079193


* Renaming features

In [82]:
# Channels
cols = list(signal.columns)
cols.remove('time')

# Topological features
feat = ['perEnt', 'numPoi', 'bottlD', 'wassD', 'landSc', 'perImg']
feat = [f + '_h' + str(hdimension) for f in feat]

# Renamed features
cols = [col + '_' + f for col in cols for f in feat]
df.columns = cols

* Adding target column 

In [84]:
df['target'] = data['target'].values

df.loc[df['target']=='trauma', 'target'] = 1
df.loc[df['target']=='healthy', 'target'] = 0

* Saving features

In [87]:
df.to_csv(f'tda_data/datar125Hz_f_1_50Hz_h{hdimension}.csv', index=False)